In [1]:
from importlib import reload
from load_cluster_data import load_cluster_data
import gc

import numpy as np
import torch
import pandas as pd 
import seaborn as sns
import collections

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

float_type = { 
    "device" : device, 
    "dtype" : torch.float, # save memory
}

hypers = {
    "eta" : 1., 
    "alpha_prior" : 1., # karin had 0.65 
    "pi_prior" : 1.
}

K = 10

import plotnine as p9
import scipy.sparse as sp
import matplotlib.pyplot as plt 
import splicing_PCA_utils
from nuclear_norm_PCA import sparse_sum

from pca_kmeans_init import pca_kmeans_init
from betabinomo_LDA_singlecells_kinit import *
import betabinomo_LDA_singlecells_kinit
reload(betabinomo_LDA_singlecells_kinit)


cpu


/gpfs/commons/home/kisaev/.local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


<module 'betabinomo_LDA_singlecells_kinit' from '/gpfs/commons/home/kisaev/leafcutter-sc/src/beta-binomial-lda/betabinomo_LDA_singlecells_kinit.py'>

### Load data

In [3]:
input_file = '/gpfs/commons/groups/knowles_lab/Karin/parse-pbmc-leafcutter/leafcutter/junctions/PBMC_input_for_LDA.h5'

In [4]:
summarized_data = pd.read_hdf(input_file, 'df')
summarized_data.head()

,cell_id,Cluster,Cluster_Counts,junction_id,gene_name,junc_count,score,junc_ratio,cell_id_index,junction_id_index,cell_type
0,B_10_10_3,371,1,1_29039426_29053106,EPB41,1,150,1.0,0,18002,B
1,B_10_10_3,509,1,1_43422225_43422479,SZT2,1,82,1.0,0,18599,B
2,B_10_10_3,576,2,1_1709133_1709517,CDK11A,1,206,0.5,0,17445,B
3,B_10_10_3,576,2,1_1709640_1712263,CDK11A,1,1245,0.5,0,17446,B
4,B_10_10_3,763,1,1_19124690_19126445,UBR4,1,270,1.0,0,17503,B


In [5]:
print(len(summarized_data.cell_id.unique())) # num cells 
print(len(summarized_data.junction_id.unique())) # num junctions
print(len(summarized_data.Cluster.unique())) # num Cluster

65457
38802
11585


In [ ]:
input_file = '/gpfs/commons/groups/knowles_lab/Karin/parse-pbmc-leafcutter/leafcutter/junctions/PBMC_input_for_LDA.h5'
final_data, coo_counts_sparse, coo_cluster_sparse, cell_ids_conversion, junction_ids_conversion = load_cluster_data(
    input_file) # , celltypes = ["B", "MemoryCD4T"])

# still do preprocessing in scipy
indices = (final_data.cell_id_index, final_data.junction_id_index)
indices_np = np.stack(indices)
junc_counts = sp.coo_matrix((final_data.junc_count, indices))
cluster_counts = sp.coo_matrix((final_data.cluster_count, indices))

In [ ]:
junc_counts_sub, cluster_counts_sub, to_keep = splicing_PCA_utils.filter_junctions(
    junc_counts, cluster_counts, plot = True)

junc_ind_keep = junction_ids_conversion["junction_id_index"][to_keep]
final_data = final_data[final_data.junction_id_index.isin(junc_ind_keep)]

In [ ]:
junction_ids_conversion=junction_ids_conversion[to_keep]
junction_ids_conversion["new_junction_id_index"] = np.arange(junction_ids_conversion.shape[0])

In [ ]:
# re-order the remaining junctions 
final_data = final_data.merge(junction_ids_conversion, on = "junction_id_index")
final_data

In [ ]:
print(junc_counts_sub.toarray()[3, 720]) # should be 3 sanity check of data 
print(cluster_counts_sub.toarray()[3, 720]) # should be 5 sanity check of data 

### Simulate Data

In [ ]:
sim_junc_counts = splicing_PCA_utils.simulate_junc_counts(
    cluster_counts_sub,
    cell_ids_conversion.cell_type.astype('category')
)

In [ ]:
# make dataframe using the following columsn 
sim_junc_counts_flat = pd.DataFrame({"cell_id_index": sim_junc_counts.row, "new_junction_id_index": sim_junc_counts.col, "new_junc_count": sim_junc_counts.data})

# update junction counts in final_data object to be the simulated counts 
final_data = final_data.merge(sim_junc_counts_flat, on = ["cell_id_index", "new_junction_id_index"])

### Prep simulated data as input into LDA 

In [ ]:
sim_data = final_data.copy() # make a copy of the data
sim_data.drop(columns = ["junc_count", "junction_id_index"], inplace = True) # drop the old junction counts and junction id index
# rename columns new_junction_id_index and new_junc_count to junction_id_index and junc_count
sim_data.rename(columns = {"new_junction_id_index": "junction_id_index", "new_junc_count": "junc_count"}, inplace = True)

In [ ]:
my_data = make_torch_data(sim_data, **float_type)

In [ ]:
my_data

### Run LDA

In [ ]:
num_trials = 1 # can't currently run more than 1 or overflow GPU memory :( 
num_iters = 300 # should also be an argument that gets fed in

# loop over the number of trials (for now just testing using one trial but in general need to evaluate how performance is affected by number of trials)
for t in range(num_trials):

    # run coordinate ascent VI
    print(K)

    ALPHA_f, PI_f, GAMMA_f, PHI_f, elbos_all = calculate_CAVI(K, my_data, float_type, hypers = hypers, num_iterations = num_iters)
    elbos_all = np.array(elbos_all)
    juncs_probs = ALPHA_f / (ALPHA_f+PI_f)
    
    theta_f = GAMMA_f / GAMMA_f.sum(1,keepdim=True)
    theta_f_plot = pd.DataFrame(theta_f.cpu())
    theta_f_plot['cell_id'] = cell_ids_conversion["cell_type"].to_numpy()
    theta_f_plot_summ = theta_f_plot.groupby('cell_id').mean()
    print(theta_f_plot_summ)
    
    # plot ELBO
    plt.plot(elbos_all[2:]); plt.show()

In [ ]:
x.shape

In [ ]:
x = theta_f.cpu().numpy()
x -= x.mean(1,keepdims=True)
x /= x.std(1,keepdims=True)
plt.hist(x.flatten(),100)
pd.crosstab( cell_ids_conversion["cell_type"], x.argmax(axis=1) )

In [ ]:
x

In [ ]:
x.shape

In [ ]:
labels=cell_ids_conversion["cell_type"].to_numpy()

In [ ]:
xdf=pd.DataFrame(x)

In [ ]:
from sklearn.decomposition import PCA
pca_dat = PCA(n_components=10)
principalComponents_breast = pca_dat.fit_transform(xdf)

In [ ]:
principal_x_Df = pd.DataFrame(data = principalComponents_breast
             , columns = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10'])
print('Explained variation per principal component: {}'.format(pca_dat.explained_variance_ratio_))


In [ ]:
principal_x_Df['cell_type'] = labels
principal_x_Df

In [ ]:
sns.scatterplot("pc1", "pc2", data=principal_x_Df, hue = "cell_type")

In [ ]:
sns.scatterplot("pc2", "pc3", data=principal_x_Df, hue = "cell_type")

In [ ]:
sns.scatterplot("pc1", "pc3", data=principal_x_Df, hue = "cell_type")

In [ ]:
# get confusion matrix type object to see if simulated data is getting "clustered" by cell type 
# via learned cell states